In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
import warnings
warnings.filterwarnings('ignore')


In [ ]:
train_df = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv',header = None)
test_df = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv',header = None)

In [ ]:
train_df[187].value_counts()

In [ ]:
# representation of classes % wise

per_class = train_df[187].value_counts()
plt.figure(figsize = (20,10))
my_circle = plt.Circle((0,0),0.7,color = 'white')
plt.pie(per_class, labels = ['normal beat','unknown Beats','Ventricular ectopic beats','Supraventricular ectopic beats','Fusion Beats'],colors = ['tab:blue','tab:orange','tab:purple','tab:olive','tab:green'],autopct = '%1.1f%%')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

Resampling for balancing dataset

In [ ]:
from sklearn.utils import resample

df_1 = train_df[train_df[187] == 1]
df_2 = train_df[train_df[187] == 2]
df_3= train_df[train_df[187] == 3]
df_4 = train_df[train_df[187] == 4]
df_0 = (train_df[train_df[187] == 0]).sample(n= 20000,random_state = 42)

df_1_upsample = resample(df_1 , replace = True , n_samples = 20000 , random_state = 123)
df_2_upsample = resample(df_2 , replace = True , n_samples = 20000 , random_state = 124)
df_3_upsample = resample(df_3 , replace = True , n_samples = 20000 , random_state = 125)
df_4_upsample = resample(df_4 , replace = True , n_samples = 20000 , random_state = 126)

train_df = pd.concat([df_0,df_1_upsample , df_2_upsample,df_3_upsample , df_4_upsample])

In [ ]:
# representation of classes % wise

per_class = train_df[187].value_counts()
plt.figure(figsize = (20,10))
my_circle = plt.Circle((0,0),0.7,color = 'white')
plt.pie(per_class , labels = ['normal beat','unknown beats' , 'Ventricular ectopic beats','Supraventricular sctopic beats' , 'Fusion Beats'],colors = ['tab:blue' , 'tab:orange' , 'tab:purple','tab:olive','tab:green'],autopct = '%1.1f%%')
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

CLASSES

In [ ]:
classes = train_df.groupby(187,group_keys = False).apply(lambda train_df : train_df.sample(1))

In [ ]:
# peek on classes
classes

plotting classes ecg

In [ ]:
# plotting classes ECG
plt.figure(figsize = (16,8))

#normal

plt.subplot(2,3,1)
plt.plot(classes.iloc[0,:186])
plt.title('normal beat')

# unknown
plt.subplot(2,3,2)
plt.plot(classes.iloc[1,:186])
plt.title('unknown beat')

# veb

plt.subplot(2,3,3)
plt.plot(classes.iloc[2,:186])
plt.title('Ventricular ectopic beat')

# seb

plt.subplot(2,3,4)
plt.plot(classes.iloc[3,:186])
plt.title('Supraventricular ectopic beat')

# fusion
plt.subplot(2,3,5)
plt.plot(classes.iloc[4,:186])
plt.title('Fusion Beat')

# show plot

plt.show()

signal heatmap

In [ ]:
def plot_hist(class_number , size , min_):
    # get the class data
    
    img = train_df.loc[train_df[187] == class_number].values
    
    # crop it
    
    img = img[:,min_:size]
    
    img_flatten = img.flatten()
    
    final1 = np.arange(min_ , size)
    for i in range(img.shape[0]-1):
        tempo1 = np.arange(min_,size)
        final1 = np.concatenate((final1,tempo1),axis = None)
    print(len(final1))
    print(len(img_flatten))
    plt.hist2d(final1 , img_flatten , bins = (80,80) , cmap = plt.cm.jet)
    plt.show()

In [ ]:
# Normal Beat
plt.figure(figsize = (12,4))
plt.subplot(1,2,1)
plt.plot(classes.iloc[0,:186])
plt.title('Normal beat')
plt.subplot(1,2,2)
plot_hist(0,70,5)

In [ ]:
# unknown

plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plt.plot(classes.iloc[1,:186])
plt.title('unknown Beat')
plt.subplot(1, 2, 2)  
plot_hist(1,50,5)

In [ ]:
# veb
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)  
plt.plot(classes.iloc[2,:186])
plt.title('Ventricular ectopic beat')
plt.subplot(1, 2, 2) 
plot_hist(2,60,30)

In [ ]:
# seb
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1) 
plt.plot(classes.iloc[3,:186])
plt.title('Supraventricular ectopic beat')
plt.subplot(1, 2, 2) 
plot_hist(3,60,25)

In [ ]:
# fusion
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)   
plt.plot(classes.iloc[4,:186])
plt.title('Fusion Beat')
plt.subplot(1, 2, 2) 
plot_hist(4,50,18)

pretreat

In [ ]:
def add_gaussian_noise(signal):
    noise = np.random.normal(0,0.5,186)
    return(signal + noise)

In [ ]:
tempo = classes.iloc[0,:186]
bruiter = add_gaussian_noise(tempo)


plt.subplot(2,1,1)
plt.plot(classes.iloc[0,:186])

plt.subplot(2,1,2)
plt.plot(bruiter)

plt.show()


In [ ]:
target_train = train_df[187]
target_test = test_df[187]
y_train = to_categorical(target_train)
y_test = to_categorical(target_test)

In [ ]:
X_train = train_df.iloc[:,:186].values 
X_test = test_df.iloc[:,:186].values
X_train = X_train.reshape(len(X_train),X_train.shape[1],1)
X_test = X_test.reshape(len(X_test),X_test.shape[1],1)

In [ ]:
def network(X_train , y_train , X_test , y_test):
    im_shape = (X_train.shape[1],1)
    inputs_cnn = Input(shape = (im_shape) , name = 'inputs_cnn')
    # first layer
    conv1_1 = Convolution1D(64,(6),activation = 'relu', input_shape = im_shape)(inputs_cnn)
    conv1_1 = BatchNormalization()(conv1_1)
    pool1 = MaxPool1D(pool_size = (3) , strides = (2) , padding = "same")(conv1_1)
    
    
    # second layer
    
    conv2_1 = Convolution1D(64,(3),activation = 'relu' , input_shape = im_shape)(pool1)
    conv2_1 = BatchNormalization()(conv2_1)
    pool2 = MaxPool1D(pool_size = (2) , strides = (2),padding = "same")(conv2_1)
    
    
    # third layer
    
    conv3_1 = Convolution1D(64 , (3) , activation = 'relu' , input_shape = im_shape)(pool2)
    conv3_1 = BatchNormalization()(conv3_1)
    pool3 = MaxPool1D(pool_size = (2),strides = (2),padding = "same")(conv3_1)
    
    flatten = Flatten()(pool3)
    
    # dense layer
    
    dense_end1 = Dense(64 , activation = 'relu')(flatten)
    dense_end2 = Dense(32 , activation = 'relu')(dense_end1)
    main_output = Dense(5,activation = 'softmax' , name = 'main_output')(dense_end2)
    
    model = Model(inputs = inputs_cnn , outputs = main_output)
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy',metrics = ['accuracy'] )
    
    callbacks = [EarlyStopping(monitor = 'val_loss' , patience = 8),ModelCheckpoint(filepath = 'best_model.h5',monitor = 'val_loss',save_best_only = True)]
    
    history = model.fit(X_train , y_train , epochs = 5 , callbacks = callbacks , batch_size = 32 , validation_data = (X_test,y_test))
    model.load_weights('best_model.h5')
    return(model,history)
    

In [ ]:
def evaluate_model(history,X_test , y_test , model):
    scores = model.evaluate((X_test) , y_test , verbose = 0)
    print('Accuracy: %.2f%%' % (scores[1]*100))
    print(history)
    
    # plotting epoch wise accuracy
    
    fig1 , ax_acc = plt.subplots()
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Model - Accuracy')
    plt.legend(['Training' , 'Validation'] , loc = 'lower right')
    plt.show()
    
    fig2, ax_loss = plt.subplots()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Model- Loss')
    plt.legend(['Training', 'Validation'], loc='upper right')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.show()
    target_names=['0','1','2','3','4']
    
    y_true=[]
    for element in y_test:
        y_true.append(np.argmax(element))
    prediction_proba=model.predict(X_test)
    prediction=np.argmax(prediction_proba,axis=1)
    cnf_matrix = confusion_matrix(y_true, prediction)
    
    
    
    

In [ ]:
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

model,history=network(X_train,y_train,X_test,y_test)

In [ ]:
evaluate_model(history,X_test,y_test,model)
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [ ]:
encoder = preprocessing.LabelEncoder()
encoder.fit(target_train)
Y_train = encoder.transform(target_train)
encoder.fit(target_test)
Y_test = encoder.transform(target_test)

In [ ]:
nsamples, nx, ny = X_train.shape
train_dataset = X_train.reshape((nsamples,nx*ny))
nsamples, nx, ny = X_test.shape
test_dataset = X_test.reshape((nsamples,nx*ny))

In [ ]:
accu = []
prec = []
recc = []
f1   = []
models =[]

In [ ]:
ytest = Y_test

In [ ]:
dtree_model = DecisionTreeClassifier(max_depth = 100).fit(train_dataset, Y_train)
dt_pred = dtree_model.predict(test_dataset)
  
# creating a confusion matrix
#cm = confusion_matrix(Y_test, dtree_predictions)


accuracy_score(Y_test, dt_pred)#, normalize=False)

In [ ]:
print('Accuracy Score : ' + str(accuracy_score(ytest, dt_pred)))
print('Precision Score : ' + str(precision_score(ytest,dt_pred,average='macro')))
print('Recall Score : ' + str(recall_score(ytest,dt_pred,average='macro')))
print('F1 Score : ' + str(f1_score(ytest,dt_pred,average='macro')))

# Classifier Confusion matrix

print('Confusion Matrix : \n' + str(confusion_matrix(ytest,dt_pred)))


accu.append(accuracy_score(ytest, dt_pred))
prec.append(precision_score(ytest,dt_pred,average='macro'))
recc.append(recall_score(ytest,dt_pred,average='macro'))
f1.append(f1_score(ytest,dt_pred,average='macro'))
models.append("Decision Tree" )